<a href="https://colab.research.google.com/github/longlinh/FitVids.js/blob/master/cfcm_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
# https://archive.ics.uci.edu/dataset/53/iris
def get_data_uci_by_url(url: str = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                        columns: list = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']) -> np.ndarray:
    _data = pd.read_csv(url, header=0, names=columns)
    # print(_data.metadata)  # Hiển thị thông tin về bộ dữ liệu
    # print(_data.variables)  # Hiển thị thông tin về các biến trong bộ dữ liệu
    return _data.iloc[:, :-1].values  # Trích xuất ma trận đặc trưng X (loại trừ nhãn lớp)

def cfcm(X: np.ndarray, W: np.ndarray, C: int, m: int = 2, epsilon: float = 1e-5) -> tuple:
    """
    Thuật toán Collaborative Fuzzy C-Means (CFCM)
    Args:
        X: Ma trận dữ liệu (N điểm dữ liệu, D thuộc tính).
        W: Ma trận trọng số hợp tác (N x N).
        C: Số lượng cụm.
        m: Tham số mờ (m>1).
        epsilon: Ngưỡng điều kiện dừng.
    Returns:
        U: Ma trận thuộc tính mờ (N x C).
        V: Ma trận tâm cụm (C x D).
    """
    N, D = X.shape
    U = np.random.rand(N, C)
    U = U / np.sum(U, axis=1, keepdims=True)
    V = np.random.rand(C, D)
    U_old = np.zeros((N, C))

    while True:
        # Cập nhật tâm cụm
        for j in range(C):
            V[j] = np.sum((W @ X) * (U[:, j]**m)[:, None], axis=0) / np.sum(W * U[:, j]**m)
        # Cập nhật ma trận thuộc tính mờ
        for i in range(N):
            for j in range(C):
                _denom = np.sum([(np.linalg.norm(X[i] - V[j]) / np.linalg.norm(X[i] - V[k]))**(2/(m-1)) for k in range(C)])
                U[i, j] = 1 / _denom
        # Kiểm tra điều kiện dừng
        if np.max(np.abs(U - U_old)) < epsilon:
            break
        U_old = U.copy()
    return U, V

def get_membership(new_points: list) -> np.ndarray:
    new_point = np.array(new_points)
    distances = np.linalg.norm(new_point - V, axis=1)
    membership = 1 / distances
    membership = membership / membership.sum()
    print("Xác suất thuộc về từng cụm của điểm dữ liệu mới:")
    return membership

In [4]:
if __name__ == "__main__":
    import time
    # Sử dụng với bộ dữ liệu Iris
    X = get_data_uci_by_url(url='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                            columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
    # Tạo ma trận trọng số W là ma trận đơn vị
    W = np.identity(X.shape[0])
    _start_time = time.time()
    C = 3  # Số lượng cụm
    # C = 7  # Số lượng cụm
    U, V = cfcm(X, W, C)
    # --------------------------------
    print("Ma trận thuộc tính mờ:", U)
    print("Ma trận tâm cụm:", V)
    print('Time request', time.time()-_start_time)
    # --------------------------------
    _start_time = time.time()
    print(get_membership(new_points=[5.5, 3.0, 4.5, 1.5]))
    print("Thời gian tính toán: ", time.time() - _start_time)

Ma trận thuộc tính mờ: [[1.57235264e-02 7.08259198e-03 9.77193882e-01]
 [1.33101060e-02 6.20642479e-03 9.80483469e-01]
 [2.15863662e-02 9.71449673e-03 9.68699137e-01]
 [4.24562643e-03 1.99575394e-03 9.93758620e-01]
 [4.60464249e-02 2.11939909e-02 9.32759584e-01]
 [1.38738663e-02 6.44554350e-03 9.79680590e-01]
 [2.71894966e-04 1.23265153e-04 9.99604840e-01]
 [4.65731294e-02 2.13821101e-02 9.32044761e-01]
 [1.11232721e-02 4.97922734e-03 9.83897501e-01]
 [2.25994414e-02 1.05979243e-02 9.66802634e-01]
 [5.21600540e-03 2.33500698e-03 9.92448988e-01]
 [1.92437114e-02 8.75224344e-03 9.72004045e-01]
 [5.11048063e-02 2.49053874e-02 9.23989806e-01]
 [7.39569379e-02 3.83348687e-02 8.87708193e-01]
 [1.06135026e-01 5.52412076e-02 8.38623766e-01]
 [3.68542961e-02 1.81254570e-02 9.45020247e-01]
 [2.62409933e-03 1.21430715e-03 9.96161594e-01]
 [6.66308891e-02 3.08094977e-02 9.02559613e-01]
 [1.51014606e-02 7.09866582e-03 9.77799874e-01]
 [2.18563991e-02 9.54278471e-03 9.68600816e-01]
 [1.11392556e-02 